In [176]:
import base64
from typing import Any
import math
import random
from sympy.ntheory import pollard_rho
from tabulate import tabulate
from functools import lru_cache
from sympy import factorint
import time

In [177]:
# coder to copypaste
class Coder:
    @staticmethod
    def encode(text: str) -> list[int]:
        return [ord(i) for i in text]

    @staticmethod
    def decode(encoded_text: list[int]) -> str:
        return "".join([chr(i) for i in encoded_text])
    
    @staticmethod
    def encode_blocks(blocks: list[int]) -> Any:
        return blocks
    
    @staticmethod
    def decode_blocks(cipher: Any) -> list[int]:
        return cipher

In [178]:
%run rsa_coder.ipynb

In [179]:
def generate_prime(order: int) -> int:
    """Генерация простого числа заданной битности"""
    while True:
        p = random.randint(10**(order-1), 10**order)
        # Убедимся, что число нечетное и достаточно большое
        if pollard_rho(p) is None:
            return p


In [180]:
def extended_gcd(a, b):
    """Расширенный алгоритм Евклида"""
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def mod_inverse(a, m):
    """Нахождение обратного элемента по модулю"""
    gcd, x, _ = extended_gcd(a, m)
    if gcd != 1:
        raise ValueError("Обратный элемент не существует")
    return x % m

In [181]:
class RSA:
    def __init__(
        self,
        key_order: int = 12,
        alphabet_length: int = 2000,
        open_key: int | None = None,
        secret_key: int | None = None,
        n: int | None = None,
        with_logging: bool = False
    ):
        self.with_logging = with_logging
        self.alphabet_length = alphabet_length

        """Инициализация RSA с генерацией ключей"""
        # Генерируем два простых числа
        if open_key:
            self.open_key = open_key
            self.n = n
        if secret_key:
            self.secret_key = secret_key
            self.n = n

        if open_key is None and secret_key is None:
            self.p = generate_prime(key_order // 2)
            self.q = generate_prime(key_order // 2)
            self.n = self.p * self.q
            # Вычисляем функцию Эйлера
            self.phi = (self.p - 1) * (self.q - 1)
            self.open_key = 65537
            while math.gcd(self.open_key, self.phi) != 1:
                self.open_key += 2
            self.secret_key = mod_inverse(self.open_key, self.phi)

        self.n_bit_length = self.n.bit_length()


    def encrypt_block(self, block: int):
        """Шифрование сообщения"""

        if block >= self.n:
            raise ValueError("Сообщение слишком большое для шифрования")
        
        return pow(block, self.open_key, self.n)
    
    def decrypt_block(self, cipherblock: int):
        """Дешифрование сообщения"""
        block = pow(cipherblock, self.secret_key, self.n)
        return block
    
    @lru_cache(None)
    def create_block(self, numbers: list[int]) -> int:
        block = 0
        for num in numbers:
            block = block * self.alphabet_length
            block += num
        return block
    
    
    def numbers_to_blocks(self, numbers: list[int]) -> list[int]:
        blocks = []
        current_nums = []
        for num in numbers:
            if self.create_block(tuple(current_nums+[num])) >= (self.n):
                if self.with_logging:
                    print(
                        f"'{"".join([chr(num) for num in current_nums])}'",
                        current_nums,
                        self.create_block(tuple(current_nums)),
                        self.encrypt_block(self.create_block(tuple(current_nums)))
                    )
                if block := self.create_block(tuple(current_nums)):
                    blocks.append(block)
                current_nums = [num]
            else:
                current_nums.append(num)
        else:
            if current_nums:
                if self.with_logging:
                    print(
                        f"'{"".join([chr(num) for num in current_nums])}'",
                        current_nums,
                        self.create_block(tuple(current_nums)),
                        self.encrypt_block(self.create_block(tuple(current_nums))),
                    )
                if block := self.create_block(tuple(current_nums)):
                    blocks.append(block)
        return blocks[::-1]

    
    def block_to_numbers(self, block: int) -> list[int]:
        numbers: list[int] = []
        while True:
            numbers.append(block % self.alphabet_length)
            block //= self.alphabet_length
            if block <= 0:
                break
        return numbers
    
    def base_encrypt(self, text: str) -> Any:
        numbers = Coder.encode(text)
        if max(numbers) >= self.alphabet_length:
            raise Exception(f"TOO LOW ALPHABET LENGTH (max alph {max(numbers)=} symbol {chr(max(numbers))})")
        blocks = self.numbers_to_blocks(numbers)

        for i in range(len(blocks)):
            blocks[i] = self.encrypt_block(blocks[i])
        return blocks
    
    def base_decrypt(self, encrypted_blocks: list[int]) -> str:
        numbers = []
        for i in range(len(encrypted_blocks)):
            decrypted_block = self.decrypt_block(encrypted_blocks[i])
            new_numbers = self.block_to_numbers(decrypted_block)
            numbers += new_numbers
        numbers = numbers[::-1]
        return Coder.decode(numbers)
    

    def encrypt(self, text: str) -> Any:
        numbers = Coder.encode(text)
        if max(numbers) >= self.alphabet_length:
            raise Exception(f"TOO LOW ALPHABET LENGTH (max alph {max(numbers)=} symbol {chr(max(numbers))})")
        # blocks = self.numbers_to_blocks(numbers)
        blocks = numbers
        print("незашифрованное сообщения (блоки): ", blocks)
        for i in range(len(blocks)):
            blocks[i] = self.encrypt_block(blocks[i])
        print("зашифрованное сообщения (блоки): ", blocks)
        cipher = Coder.encode_blocks(blocks)
        return cipher

    def decrypt(self, cipher: Any) -> str:
        blocks = Coder.decode_blocks(cipher)
        numbers = []
        for i in range(len(blocks)):
            decrypted_block = self.decrypt_block(blocks[i])
            # new_numbers = self.block_to_numbers(decrypted_block)
            new_numbers = [decrypted_block]
            syms = "".join([chr(num) for num in new_numbers])
            print(blocks[i], decrypted_block, new_numbers, f"'{syms}'")
            numbers += new_numbers
        numbers = numbers[::-1]
        return Coder.decode(numbers)


In [182]:
alphabet_length = 1200
key_order = 8
message = "ответ is 1 :)="

# alphabet_length = int(input("Введите длину алфавита (1500 хватит на многое): "))
# key_order = int(input("Введите количество знаков в ключах: "))
# message = input("Введите фразу для шифрования: ")

rsa = RSA(key_order=key_order, alphabet_length=alphabet_length, with_logging=True)
table = [
    ["простое число P", rsa.p],
    ["простое число Q", rsa.q],
    ["модуль N=P*Q", rsa.n],
    ["функция Эйлера", rsa.phi],
    ["открытый ключ e", rsa.open_key],
    ["открытый ключ d", rsa.secret_key],
]
print(
    tabulate(
        table,
        headers=["Параметр", "Значение"],
        tablefmt="grid",
    )
)
print("\n")
print(f"сообщение для шифрования: {message}")
crypted_message = rsa.encrypt(message)
print(f"сообщение после полного цикла шифрования: {crypted_message}")
decrypted_message = rsa.decrypt(crypted_message)
print(f"сообщение после полного цикла расшифрования: {decrypted_message}")



+-----------------+------------+
| Параметр        |   Значение |
+=================+============+
| простое число P |       6673 |
+-----------------+------------+
| простое число Q |       1481 |
+-----------------+------------+
| модуль N=P*Q    |    9882713 |
+-----------------+------------+
| функция Эйлера  |    9874560 |
+-----------------+------------+
| открытый ключ e |      65537 |
+-----------------+------------+
| открытый ключ d |    7261313 |
+-----------------+------------+


сообщение для шифрования: ответ is 1 :)=
незашифрованное сообщения (блоки):  [1086, 1090, 1074, 1077, 1090, 32, 105, 115, 32, 49, 32, 58, 41, 61]
зашифрованное сообщения (блоки):  [8777875, 7788584, 1403295, 9095019, 7788584, 6422983, 378841, 6622745, 6422983, 8612874, 6422983, 6497158, 2308616, 9540107]
сообщение после полного цикла шифрования: ODc3Nzg3NSA3Nzg4NTg0IDE0MDMyOTUgOTA5NTAxOSA3Nzg4NTg0IDY0MjI5ODMgMzc4ODQxIDY2MjI3NDUgNjQyMjk4MyA4NjEyODc0IDY0MjI5ODMgNjQ5NzE1OCAyMzA4NjE2IDk1NDAxMDc=
877787

In [183]:
rsa.with_logging = False

cipher_blocks = Coder.decode_blocks(crypted_message)

print("ПОШАГОВАЯ ПРОВЕРКА: ДЛЯ КАЖДОГО БЛОКА ИЩЕМ M, ТАКОЕ ЧТО M^e mod N = C")

min_char, max_char = 0, rsa.alphabet_length - 1

print(f"Диапазон числовых кодов символов в сообщении: {min_char} - {max_char}")

# Для каждого зашифрованного блока пытаемся найти M
found_blocks = []
total_tested = 0

for block_idx, C in enumerate(cipher_blocks):
    print(f"{'='*40}")
    print(f"БЛОК {block_idx}: C = {C}")
    
    #  Пытаемся найти M прямым перебором для небольших значений
    # (это работает только для очень маленьких сообщений)
    
    for test_M in range(rsa.n):
        total_tested += 1
        # print(f"проверяем M={test_M}: {test_M}^{rsa.open_key} mod {rsa.n} = {pow(test_M, rsa.open_key, rsa.n)}")
        if pow(test_M, rsa.open_key, rsa.n) == C:
            print(f"✓ Найдено! M = {test_M}")
            print(f"Проверка: {test_M}^{rsa.open_key} mod {rsa.n} = {C}")
            found_blocks.append(test_M)
            found = True
            break

print("\n" + "="*80)
print("РЕЗУЛЬТАТЫ:")
print("="*80)

if all(b is not None for b in found_blocks):
    print("✓ Все блоки успешно найдены!")
    print(f"Найденные блоки M: {found_blocks}")
    
    # Восстанавливаем сообщение из найденных блоков
    recovered_numbers = []
    for block in found_blocks:
        recovered_numbers += rsa.block_to_numbers(block)
    
    # numbers_to_blocks возвращает блоки в обратном порядке, поэтому переворачиваем
    recovered_numbers = recovered_numbers[::-1]
    
    
    recovered_message = Coder.decode(recovered_numbers)
    print(f"Восстановленное сообщение: '{recovered_message[::-1]}'")
    print(f"Совпадает с исходным: {recovered_message == message}")
else:
    print("✗ Не все блоки удалось расшифровать")
    print(f"Успешно расшифровано блоков: {sum(1 for b in found_blocks if b is not None)}/{len(found_blocks)}")

print(f"Всего протестировано комбинаций: {total_tested}")


ПОШАГОВАЯ ПРОВЕРКА: ДЛЯ КАЖДОГО БЛОКА ИЩЕМ M, ТАКОЕ ЧТО M^e mod N = C
Диапазон числовых кодов символов в сообщении: 0 - 1199
БЛОК 0: C = 8777875
✓ Найдено! M = 1086
Проверка: 1086^65537 mod 9882713 = 8777875
БЛОК 1: C = 7788584
✓ Найдено! M = 1090
Проверка: 1090^65537 mod 9882713 = 7788584
БЛОК 2: C = 1403295
✓ Найдено! M = 1074
Проверка: 1074^65537 mod 9882713 = 1403295
БЛОК 3: C = 9095019
✓ Найдено! M = 1077
Проверка: 1077^65537 mod 9882713 = 9095019
БЛОК 4: C = 7788584
✓ Найдено! M = 1090
Проверка: 1090^65537 mod 9882713 = 7788584
БЛОК 5: C = 6422983
✓ Найдено! M = 32
Проверка: 32^65537 mod 9882713 = 6422983
БЛОК 6: C = 378841
✓ Найдено! M = 105
Проверка: 105^65537 mod 9882713 = 378841
БЛОК 7: C = 6622745
✓ Найдено! M = 115
Проверка: 115^65537 mod 9882713 = 6622745
БЛОК 8: C = 6422983
✓ Найдено! M = 32
Проверка: 32^65537 mod 9882713 = 6422983
БЛОК 9: C = 8612874
✓ Найдено! M = 49
Проверка: 49^65537 mod 9882713 = 8612874
БЛОК 10: C = 6422983
✓ Найдено! M = 32
Проверка: 32^65537 mod 9

In [184]:
rsa.with_logging = False

# Добавляем функцию факторизации (для учебных целей)
def factorize_n(n, max_iterations=1000000):
    """Простейшая факторизация пробным делением (для учебных целей)"""
    print(f"Начинаем факторизацию N = {n}...")
    print(f"Битовая длина N: {n.bit_length()} бит")
    
    factors = []
    temp_n = n
    
    # Проверяем делимость на маленькие простые числа
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
    
    print("\nПроверка делимости на маленькие простые числа:")
    for p in small_primes:
        count = 0
        while temp_n % p == 0:
            temp_n //= p
            count += 1
        if count > 0:
            factors.append((p, count))
            print(f"  Найден множитель: {p}^{count}")
    
    # Если осталось составное число, пробуем факторизовать дальше
    if temp_n > 1:
        print(f"\nОстаток для факторизации: {temp_n}")
        
        # Пробное деление до квадратного корня
        limit = min(int(math.isqrt(temp_n)) + 1, max_iterations)
        print(f"Проверяем делители до {min(limit, 10000)}...")
        
        found = False
        for i in range(2, limit, 1):
            if i % 10000 == 0 and i > 0:
                print(f"  Проверено {i} делителей...")
            
            if temp_n % i == 0:
                factors.append((i, 1))
                factors.append((temp_n // i, 1))
                print(f"  Найден делитель: {i}")
                print(f"  Второй делитель: {temp_n // i}")
                found = True
                break
        
        if not found and temp_n > 1:
            # Если не нашли делителей, значит temp_n простое
            factors.append((temp_n, 1))
            print(f"  Оставшийся множитель {temp_n} - простое число")
    
    # Восстанавливаем p и q
    if len(factors) >= 2:
        # Для RSA нам нужны ровно два множителя
        p = factors[0][0]
        q = 1
        for factor, power in factors[1:]:
            q *= factor ** power
        if p * q == n:
            return p, q
    
    # Альтернативный подход: если не удалось, используем sympy
    print("\nИспользуем более продвинутый алгоритм факторизации (sympy)...")
    try:
        factor_dict = factorint(n)
        factors = list(factor_dict.items())
        if len(factors) == 2:
            p = factors[0][0]
            q = factors[1][0]
            if factors[0][1] > 1:
                p = factors[0][0] ** factors[0][1]
            if factors[1][1] > 1:
                q = factors[1][0] ** factors[1][1]
            return p, q
    except:
        pass
    
    return None, None


print(f"сообщение после полного цикла шифрования: {crypted_message}")
print("\n" + "="*100)
print("ВЗЛОМ RSA ЧЕРЕЗ ФАКТОРИЗАЦИЮ МОДУЛЯ N")
print("="*100)

# Шаг 1: Получаем открытый ключ (e, N) и зашифрованные блоки
print("1. ИНФОРМАЦИЯ, ДОСТУПНАЯ АТАКУЮЩЕМУ:")
print("-"*50)

cipher_blocks = Coder.decode_blocks(crypted_message)

table_public_info = [
    ["Открытый ключ e", rsa.open_key],
    ["Модуль N", rsa.n],
    ["Битовая длина N", f"{rsa.n.bit_length()} бит"],
    ["Количество зашифрованных блоков", len(cipher_blocks)],
    ["Зашифрованные блоки (C)", cipher_blocks]
]

print(tabulate(table_public_info, headers=["Параметр", "Значение"], tablefmt="grid"))

print()
# Шаг 2: Таблица шифрования (как шифровалось сообщение)
print("2. ТАБЛИЦА ШИФРОВАНИЯ СООБЩЕНИЯ:")
print("-"*50)

numbers = Coder.encode(message)
blocks_before_encryption = rsa.numbers_to_blocks(numbers)

table_encryption = []
for i, (block, cipher) in enumerate(zip(blocks_before_encryption, cipher_blocks)):
    numbers_in_block = rsa.block_to_numbers(block)
    symbols = ''.join([chr(n) for n in numbers_in_block])
    
    table_encryption.append([
        i + 1,
        f"'{symbols}'",
        numbers_in_block,
        block,
        f"{block}^{rsa.open_key} mod {rsa.n}",
        cipher
    ])

print(tabulate(table_encryption, 
               headers=["Блок", "Символы", "Коды", "M", "Формула", "C = M^e mod N"], 
               tablefmt="grid"))

print()
# Шаг 3: Факторизация модуля N
print("3. ФАКТОРИЗАЦИЯ МОДУЛЯ N:")
print("-"*50)

start_time = time.time()
p, q = factorize_n(rsa.n)
factoring_time = time.time() - start_time

if p and q:
    print("✓ Факторизация успешна!")
    print("  Найденные простые множители:")
    print(f"  p = {p}")
    print(f"  q = {q}")
    print(f"  Проверка: p * q = {p} * {q} = {p * q}")
    print(f"  Совпадает с N: {p * q == rsa.n}")
    print(f"  Время факторизации: {factoring_time:.4f} секунд")
else:
    print("✗ Факторизация не удалась")
    print(f"  Время затрачено: {factoring_time:.4f} секунд")
    # Для демонстрации используем настоящие p и q (в реальной атаке их не было бы)
    print("  (Для демонстрации используем настоящие p и q из RSA объекта)")
    p, q = rsa.p, rsa.q

print()
# Шаг 4: Восстановление секретного ключа
print("4. ВОССТАНОВЛЕНИЕ СЕКРЕТНОГО КЛЮЧА:")
print("-"*50)

# Вычисляем φ(N) = (p-1)*(q-1)
phi_recovered = (p - 1) * (q - 1)

# Находим d = e^(-1) mod φ(N)
# Используем расширенный алгоритм Евклида
def egcd(a, b):
    if b == 0:
        return (a, 1, 0)
    else:
        g, x, y = egcd(b, a % b)
        return (g, y, x - (a // b) * y)

g, x, y = egcd(rsa.open_key, phi_recovered)
if g == 1:
    d_recovered = x % phi_recovered
    print("✓ Секретный ключ успешно восстановлен!")
    
    table_key_recovery = [
        ["p", p],
        ["q", q],
        ["φ(N) = (p-1)*(q-1)", f"({p}-1)*({q}-1) = {phi_recovered}"],
        ["Открытый ключ e", rsa.open_key],
        ["Обратный элемент d = e^(-1) mod φ(N)", f"{rsa.open_key}^(-1) mod {phi_recovered} = {d_recovered}"],
        ["Проверка: e*d mod φ(N)", f"{rsa.open_key}*{d_recovered} mod {phi_recovered} = {(rsa.open_key * d_recovered) % phi_recovered}"]
    ]
    
    print(tabulate(table_key_recovery, headers=["Шаг", "Результат"], tablefmt="grid"))
    
    # Проверяем, совпадает ли с настоящим секретным ключом
    print(f"\n  Сравнение с настоящим секретным ключом:")
    print(f"  Восстановленный d: {d_recovered}")
    # print(f"  Настоящий d: {rsa.secret_key}")
    # print(f"  Совпадают: {d_recovered == rsa.secret_key}")
else:
    print("✗ Не удалось восстановить секретный ключ: e и φ(N) не взаимно просты")
    d_recovered = None

# Шаг 5: Расшифрование сообщения с восстановленным ключом
print("\n\n5. РАСШИФРОВАНИЕ СООБЩЕНИЯ С ВОССТАНОВЛЕННЫМ КЛЮЧОМ:")
print("-"*50)

if d_recovered:
    print("Используем восстановленный секретный ключ для расшифрования...")
    
    # Создаем временный RSA объект с восстановленным ключом
    temp_rsa = RSA(
        alphabet_length=alphabet_length,
        open_key=rsa.open_key,
        secret_key=d_recovered,
        n=rsa.n,
        with_logging=False
    )
    
    # Расшифровываем сообщение
    decrypted_message = temp_rsa.decrypt(crypted_message)
    
    print(f"\n✓ Сообщение успешно расшифровано!")
    print(f"  Расшифрованное сообщение: '{decrypted_message}'")
    print(f"  Совпадает с исходным: '{decrypted_message == message}'")
    
    # Таблица расшифрования
    print("\n  Таблица расшифрования блоков:")
    
    table_decryption = []
    for i, cipher_block in enumerate(cipher_blocks):
        # Расшифровываем блок
        M_recovered = pow(cipher_block, d_recovered, rsa.n)
        numbers_in_block = temp_rsa.block_to_numbers(M_recovered)
        symbols = ''.join([chr(n) for n in numbers_in_block])
        
        table_decryption.append([
            i + 1,
            cipher_block,
            f"{cipher_block}^{d_recovered} mod {rsa.n}",
            M_recovered,
            numbers_in_block,
            f"'{symbols}'"
        ])
    
    print(tabulate(table_decryption, 
                   headers=["Блок", "C", "Формула", "M = C^d mod N", "Коды символов", "Символы"], 
                   tablefmt="grid"))
    
else:
    print("✗ Невозможно расшифровать без секретного ключа")


сообщение после полного цикла шифрования: ODc3Nzg3NSA3Nzg4NTg0IDE0MDMyOTUgOTA5NTAxOSA3Nzg4NTg0IDY0MjI5ODMgMzc4ODQxIDY2MjI3NDUgNjQyMjk4MyA4NjEyODc0IDY0MjI5ODMgNjQ5NzE1OCAyMzA4NjE2IDk1NDAxMDc=

ВЗЛОМ RSA ЧЕРЕЗ ФАКТОРИЗАЦИЮ МОДУЛЯ N
1. ИНФОРМАЦИЯ, ДОСТУПНАЯ АТАКУЮЩЕМУ:
--------------------------------------------------
+---------------------------------+-------------------------------------------------------------------------------------------------------------------------------+
| Параметр                        | Значение                                                                                                                      |
+=================================+===============================================================================================================================+
| Открытый ключ e                 | 65537                                                                                                                         |
+-------------------------

In [185]:
alphabet_length = 96

# alphabet_length = int(input("Введите длину алфавита (1500 хватит на многое): "))
# key_order = int(input("Введите количество знаков в ключах: "))
# message = input("Введите фразу для шифрования: ")

rsa = RSA(alphabet_length=96, open_key=65537, n=1617862235270391231360844114948922946768211447953631122654063354590329423, with_logging=True)
# table = [
#     ["простое число P", rsa.p],
#     ["простое число Q", rsa.q],
#     ["модуль N=P*Q", rsa.n],
#     ["функция Эйлера", rsa.phi],
#     ["открытый ключ e", rsa.open_key],
#     ["открытый ключ d", rsa.secret_key],
# ]
# print(
#     tabulate(
#         table,
#         headers=["Параметр", "Значение"],
#         tablefmt="grid",
#     )
# )
message = ""
crypted_message = "дбц#Ё$гчы%ЗыЧКЩшЛЦЪЙФФД!?цеи5В1Мъ2оеуаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйва1!8З0рВ+рд9ЩНргГлВ&Ъ&ЦБ#ЗЛляуоИА_*ЁеаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвд?Г7+к?ЖЖКс4)С)Н8СУЕрЬЧЦццч08у!ёйИы+:еК?н)еЧ!%зЕ*)6&^р7вЁ! чн#СЯЦЗыЁ4^)99эгЩоцб-сг$ЗХомжЖ&ф;а,,ТА^ЁСБ5=:Ч3жОтПГ                                     е#3Г2заф 6Ац№б;Л&Ь ШНЬ щСХМЙуйшфюэ6Д.                                     дыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ё                                    а                                     дфдтмюИе*Ъ(Ё2Ч7кщкТ(лЫтюэ№09 ,лё=ЙвъФаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвдыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ёбзк,П)6флрЦ)д#ЧшБсЫялРйГ-э_ДувАцй2фожбв 8ШОчСрёщъм%ЫпмЖблМЭЖеЦ#ЛЗьз%8щ0йИ9аЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвв?7!в№ВнОдАь3=йЙ1тКАч-зёехЯБЗЖ- ф?зш3бЖ!_б&ь8ч%Ц3.9фЖ#Д%Э?#ОД=П$ъМГ_ьРёНэщдыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ёаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйва4?аьц(,ЕЕЗБёбУ9ЧВё:.*ЭЭЪДг;фт5щ2ЭмРигЩоцб-сг$ЗХомжЖ&ф;а,,ТА^ЁСБ5=:Ч3жОтПГ                                     абр8ыц9з№ъЯЭГЦч&4хаЦдЫБ9брЬв25-9Л*НХМд8йГшПяпЩхШиЧ8еяТ:*%8ювИнЗЧЯчсЁЬИЫНА^а1!8З0рВ+рд9ЩНргГлВ&Ъ&ЦБ#ЗЛляуоИА_*Ёеб-сяьицЛЗЯьуфЖШтКм ЦШ_ыр0&,!ьШбыПДкхС                                     бв 8ШОчСрёщъм%ЫпмЖблМЭЖеЦ#ЛЗьз%8щ0йИ9                                    агЪРс #ЙНЁл2ъёж=ц9Л7-ьиуяц08№5Э;ц?МАХэдыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ё                                    а                                     еЁеЮ7с0ОЫ_;к2бФЩв1ОП$_д!!ЯвЯ-Шх05цК№ёбв 8ШОчСрёщъм%ЫпмЖблМЭЖеЦ#ЛЗьз%8щ0йИ9аЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6МйвеК?н)еЧ!%зЕ*)6&^р7вЁ! чн#СЯЦЗыЁ4^)99эв3Л.бМ7*)дЪ4Я;а9Г+ШЖ2луН--7зб1ДкзХЖс1гЪРс #ЙНЁл2ъёж=ц9Л7-ьиуяц08№5Э;ц?МАХэаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвеб6#._нёЪд(Э#ТЫ67оЬШн7з*йя-,!:КЖдЫьлЛдыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ё                                    ае#3Г2заф 6Ац№б;Л&Ь ШНЬ щСХМЙуйшфюэ6Д.                                     аЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвбзк,П)6флрЦ)д#ЧшБсЫялРйГ-э_ДувАцй2фож                                     вб2ЯчвщоъоЖ2Ё9Вс-Ц:эюЁ:$.жТд+4эЁ;О(шБбцуНк№у1бЦЛ;;0у9БщЁае№*5юиздеь9Пре*кщ"
print(f"сообщение после полного цикла шифрования: {crypted_message}")


сообщение после полного цикла шифрования: дбц#Ё$гчы%ЗыЧКЩшЛЦЪЙФФД!?цеи5В1Мъ2оеуаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйва1!8З0рВ+рд9ЩНргГлВ&Ъ&ЦБ#ЗЛляуоИА_*ЁеаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвд?Г7+к?ЖЖКс4)С)Н8СУЕрЬЧЦццч08у!ёйИы+:еК?н)еЧ!%зЕ*)6&^р7вЁ! чн#СЯЦЗыЁ4^)99эгЩоцб-сг$ЗХомжЖ&ф;а,,ТА^ЁСБ5=:Ч3жОтПГ                                     е#3Г2заф 6Ац№б;Л&Ь ШНЬ щСХМЙуйшфюэ6Д.                                     дыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ё                                    а                                     дфдтмюИе*Ъ(Ё2Ч7кщкТ(лЫтюэ№09 ,лё=ЙвъФаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвдыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ёбзк,П)6флрЦ)д#ЧшБсЫялРйГ-э_ДувАцй2фожбв 8ШОчСрёщъм%ЫпмЖблМЭЖеЦ#ЛЗьз%8щ0йИ9аЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйвв?7!в№ВнОдАь3=йЙ1тКАч-зёехЯБЗЖ- ф?зш3бЖ!_б&ь8ч%Ц3.9фЖ#Д%Э?#ОД=П$ъМГ_ьРёНэщдыз4%Ч:кНъетБ с оз3№2НЙЩ(ГЫС)_о96 д?ёаЪ4енЁЭд№Ц(Я9т№8п)тНксЛЪуу7;хЭ8Фв6Мйва4?аьц(,ЕЕЗБёбУ9ЧВё:.*ЭЭЪДг;фт5щ2ЭмРигЩоцб-сг$ЗХомжЖ&ф;а,,ТА^ЁСБ5=:Ч3жОтПГ                                 